<a href="https://colab.research.google.com/github/dbrody112/Frequentist-Machine-Learning/blob/main/Dan_Brody_Gradient_Boosting_Trees_Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Gradient Boosting Trees**

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving heart-disease.csv to heart-disease.csv


In [ ]:
import pandas as pd
import numpy as np
import scipy
from matplotlib import pyplot
import seaborn
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import make_pipeline,Pipeline
import xgboost as xgb
from sklearn.metrics import accuracy_score
import io

#LOADING DATA

dataset = pd.read_csv(io.BytesIO(uploaded['heart-disease.csv']))
#DATA VISUALIZATION WITH CORRELATION

dataset.corr()


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
age,1.000000,-0.098447,-0.068653,0.279351,0.213678,0.121308,-0.116211,-0.398522,0.096801,0.210013,-0.168814,0.276326,0.068001,-0.225439
sex,-0.098447,1.000000,-0.049353,-0.056769,-0.197912,0.045032,-0.058196,-0.044020,0.141664,0.096093,-0.030711,0.118261,0.210041,-0.280937
cp,-0.068653,-0.049353,1.000000,0.047608,-0.076904,0.094444,0.044421,0.295762,-0.394280,-0.149230,0.119717,-0.181053,-0.161736,0.433798
trestbps,0.279351,-0.056769,0.047608,1.000000,0.123174,0.177531,-0.114103,-0.046698,0.067616,0.193216,-0.121475,0.101389,0.062210,-0.144931
chol,0.213678,-0.197912,-0.076904,0.123174,1.000000,0.013294,-0.151040,-0.009940,0.067023,0.053952,-0.004038,0.070511,0.098803,-0.085239
fbs,0.121308,0.045032,0.094444,0.177531,0.013294,1.000000,-0.084189,-0.008567,0.025665,0.005747,-0.059894,0.137979,-0.032019,-0.028046
restecg,-0.116211,-0.058196,0.044421,-0.114103,-0.151040,-0.084189,1.000000,0.044123,-0.070733,-0.058770,0.093045,-0.072042,-0.011981,0.137230
thalach,-0.398522,-0.044020,0.295762,-0.046698,-0.009940,-0.008567,0.044123,1.000000,-0.378812,-0.344187,0.386784,-0.213177,-0.096439,0.421741
exang,0.096801,0.141664,-0.394280,0.067616,0.067023,0.025665,-0.070733,-0.378812,1.000000,0.288223,-0.257748,0.115739,0.206754,-0.436757
oldpeak,0.210013,0.096093,-0.149230,0.193216,0.053952,0.005747,-0.058770,-0.344187,0.288223,1.000000,-0.577537,0.222682,0.210244,-0.430696


In [ ]:
from sklearn.preprocessing import StandardScaler

#DEFINING TARGET AND PREDICTORS

X = dataset.drop(["target", "thal","fbs","sex"],axis=1 )
y = dataset["target"]

#SCALING DATA TO NORMAL DISTRIBUTION
scaler  = StandardScaler()
X_scaled = scaler.fit_transform(X)

from sklearn.model_selection import train_test_split

#SPLITTING DATA TO TRAIN, TEST, VALIDATION

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y.values, test_size = 0.2, random_state = 25, shuffle = True)
X_validation, X_test,y_validation, y_test = train_test_split(X_test,y_test, test_size = 0.5,random_state = 25, shuffle = True)

In [ ]:
#DEFINING SCORING METHOD TO PREVENT OVERFITTING USING GRIDSEARCH

def best_ntree_score(estimator, X, y):
    """
    This scorer uses the best_ntree_limit to return
    the best AUC ROC score
    """
    try:
        y_predict = estimator.predict_proba(X,ntree_limit=estimator.best_ntree_limit)
    except AttributeError:
        y_predict = estimator.predict_proba(X)
    return roc_auc_score(y, y_predict[:, 1])

In [ ]:
#CROSS VALIDATION

xgb_model = xgb.XGBClassifier()
clf = GridSearchCV(xgb_model,
                   {'max_depth': [2, 4, 6],
                    'n_estimators': [50, 100, 200,300,400,500,600,700,800,900,1000],
                    'learning_rate': np.logspace(-10,0,20),'reg_alpha':[0.001,0.1,0.2,0.3],'reg_lambda': [0.001,0.1,0.2,0.3]}, best_ntree_score)
clf.fit(X_validation, y_validation)
print(clf.best_score_)
print(clf.best_params_)

0.763888888888889
{'learning_rate': 0.2976351441631313, 'max_depth': 2, 'n_estimators': 100, 'reg_alpha': 0.001, 'reg_lambda': 0.2}


In [ ]:
learning_rate = clf.best_params_['learning_rate']
max_depth = clf.best_params_['max_depth']
n_estimators = clf.best_params_['n_estimators']
reg_alpha = clf.best_params_['reg_alpha']
reg_lambda = clf.best_params_['reg_lambda']

In [ ]:
#IMPLEMENTING XGBClassifier

clf2 = xgb.XGBClassifier(max_depth = max_depth,learning_rate = learning_rate, n_estimators = n_estimators, reg_alpha = reg_alpha, reg_lambda = reg_lambda)
clf2.fit(X_train,y_train)
pred = clf2.predict(X_test)

In [ ]:
#GETTING ACCURACY FOR CLASSIFIER AND COMPARING IT TO BENCHMARKS
print(f"actual accuracy is {accuracy_score(y_test,pred)}  while benchmark with zeros is {accuracy_score(y_test,np.zeros((len(y_test),1)))} and benchmark with ones is {accuracy_score(y_test,np.ones((len(y_test),1)))}")

actual accuracy is 0.6774193548387096  while benchmark with zeros is 0.5806451612903226 and benchmark with ones is 0.41935483870967744


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
#SAVING MODEL

model_save_name = '0001.model'
path = F"/content/gdrive/My Drive/{model_save_name}" 
clf2.save_model(path)

In [ ]:
import plotly.express as px
importances = clf2.feature_importances_
fig = px.bar(X, x=importances, y=X.columns, title = "XGBoost Feature Importance Plot",orientation ="h").update_yaxes(categoryorder = "total descending")
fig.show()

I did not use this dataset in assignment 1 but I did use it in assignment 2, however I did not do the optional L1 penalty for feature selection in that assignment. I will try that after this assignment.